In [303]:
import pandas as pd
import ast
import numpy as np

In [304]:
path = 'if_else.csv'
df = pd.read_csv(path, index_col=0)[['premise', 'hypothesis', 'prompt', 'rationale']]

In [305]:
df['hypothesis'] = df['hypothesis'].apply(ast.literal_eval)

In [307]:
df.rationale = df.rationale.astype(str)
df['split'] = df.rationale.apply(lambda x: x.lower().split('most likely answer'))
df.split = df.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [308]:
df.head(1)

,premise,hypothesis,prompt,rationale,split
0,Where could you find a person standing next to...,"[ocean, soccer game, fishing gear, fisherman's...",Questions: Where could you find a person stand...,1. The correct answer is **(B) soccer game**. ...,1. the correct answer is **(b) soccer game**. ...


In [309]:
# get correct index
df['correct_index'] = [{} for _ in range(len(df))]
for index, row in df.iterrows():
    for choice in row['hypothesis']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice.lower())

In [310]:
df['LLM_answer'] = df.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [316]:
df[df.LLM_answer.isna()]

,premise,hypothesis,prompt,rationale,split,correct_index,LLM_answer
27,Where could you find millions of jellyfish?,"[north sea, smack, sponge bob's house, store, ...",Questions: Where could you find millions of je...,nan,nan,{},NaN
28,Where could you find more than a few bald eagle?,"[the park, in washington, eagle's nest, aerie,...",Questions: Where could you find more than a fe...,"9th and 20th centuries, and they are considere...","9th and 20th centuries, and they are considere...",{},NaN
29,Where could you find multiple ruler that are o...,"[measure distance, england, office, drawer, desk]",Questions: Where could you find multiple ruler...,b,b,{},NaN
260,Joe and Jane have begun reaching a tentative a...,"[compromises, eloping, calmness, fucking, hand...",Questions: Joe and Jane have begun reaching a ...,1. The correct answer is (D) fuckin,1. the correct answer is (d) fuckin,{},NaN
261,Joe and Jim were running errands together. Th...,"[stress, aggravation, efficiency, tiredness, n...",Questions: Joe and Jim were running errands to...,nan,nan,{},NaN
...,...,...,...,...,...,...,...
9637,She always made sure her infant was sleeping t...,"[nightmares, death, dreams, erections, vaccines]",Questions: She always made sure her infant was...,nan,nan,{},NaN
9638,She always wondered how safe it was to use her...,"[jungle, drug store, box, beauty salon, bathroom]",Questions: She always wondered how safe it was...,nan,nan,{},NaN
9639,"She asked to swallow semen, he shrugged and re...","[you're into, prostitute, in a porn movie, por...","Questions: She asked to swallow semen, he shru...",nan,nan,{},NaN
9640,"She began to play violin at a young age, she h...","[inspiring, like music, be musician, make musi...",Questions: She began to play violin at a young...,2,2,{},NaN


In [290]:
# read paper's answer
paper = pd.read_csv('/home/huy/Desktop/HCMUS/distilling-step-by-step/[API] CQA/paper.csv', index_col=0)

In [299]:
consensus = pd.read_csv('/home/huy/Desktop/HCMUS/distilling-step-by-step/[API] CQA/consensus - full.csv', index_col=0)

In [291]:
paper['premise'] = paper.input.str.split('\n').apply(lambda x: x[0])
paper.set_index('premise', inplace=True)

In [292]:
df.set_index('premise', inplace=True)

KeyError: "None of ['premise'] are in the columns"

In [301]:
paper['my_label'] = consensus['LLM_answer']

In [302]:
sum(paper.label == paper.my_label) / len(paper)

0.7465352633199877

In [296]:
df.to_csv('neutral - full.csv')

In [300]:
consensus

,hypothesis,rationales,split,correct_index,LLM_answer
premise,,,,,
"""There are 10 apples on an apple tree. Three fall off. Now there are X apples."" What is this an example of?","['park', 'coloring book', 'garden center', 'ma...",1. The commonly agreed-upon answer to the ques...,1. the commonly agreed-upon answer to the ques...,{'math problem': 163},math problem
"A John is a bum. Much like the stereotype, he lives near this sort of transportation infrastructure. Where does he live?","['bus depot', 'beach', 'train station', 'bridg...",2. The commonly agreed-upon answer to the ques...,2. the commonly agreed-upon answer to the ques...,{'bridge': 179},bridge
"A bad person places little value on being honest, acting without pretense or being what?","['excellent', 'upright', 'premium', 'competent...",3. The commonly agreed-upon answer to the ques...,3. the commonly agreed-upon answer to the ques...,{'sincere': 145},sincere
"A bald eagle flies over St. Paul, where is it?","['texas', 'thermal', 'minnesota', 'canada', 'p...",4. The commonly agreed-upon answer to the ques...,4. the commonly agreed-upon answer to the ques...,{'minnesota': 103},minnesota
"A battleship is a powerful vessel. If you need something similar but faster, what would you use?","['yatch', 'corvette', 'aircraft carrier', 'des...",5. The commonly agreed-upon answer to the ques...,5. the commonly agreed-upon answer to the ques...,{'destroyer': 155},destroyer
...,...,...,...,...,...
where is the information superhighway?,"['heavily travelled area', 'cyberspace', 'indu...",7. The commonly agreed-upon answer to the ques...,7. the commonly agreed-upon answer to the ques...,{'cyberspace': 99},cyberspace
where would a good newspaper boy put a paper in the rain?,"['library', 'lawn', 'trash', 'roof', 'front do...",8. The commonly agreed-upon answer to the ques...,8. the commonly agreed-upon answer to the ques...,{'front door': 118},front door
which door with lock regularly holds cars?,"['house', 'autolock', 'garage', 'file cabinet'...",9. The commonly agreed-upon answer to the ques...,9. the commonly agreed-upon answer to the ques...,{'garage': 103},garage
